In [5]:
!pip3 install tweepy python-twitter kafka-python python-dotenv deep-translator

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import os
import tweepy
import json
from tweepy import OAuthHandler
from tweepy import Stream, StreamingClient
from kafka import KafkaProducer
from deep_translator import GoogleTranslator

In [7]:
from dotenv import load_dotenv

load_dotenv()

consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
bearer_token = os.getenv('BEARER_TOKEN')

print(consumer_key, consumer_secret, access_token, access_token_secret)

YbnSPEPRMhAEvW00ObewBZ38K AotpMI3ISxcYeXeXy6bIn58JdmB5Xyct0kFvJcOKI1SPGJ0LvJ 1232436911043760129-QKyUzUUeSRs86zMgHqrMlRjtK9ZTxg QY0racyggli9NnvgBER06GeO8dGyJMLU6Kwu4j7QScAMq


In [8]:
topic_name = 'twitter-data2'

kafka_producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Subclass Stream to print IDs of Tweets received
class IDPrinter(tweepy.StreamingClient):
    def on_tweet(self, tweet):
        translated = GoogleTranslator(source='auto', target='en').translate(tweet.data['text'])
        print(translated)
        kafka_producer.send(topic_name, translated.encode('utf-8')).get(timeout=10)

printer = IDPrinter(bearer_token)
printer.add_rules(tweepy.StreamRule("Lula -is:retweet"))
printer.filter()
printer.sample()

# Filter realtime Tweets by keyword


RT @skingotic: 10 reais the cinema signs of the squid coming back
RT @JoaoBaraldoM: Lula put Haddad as a front, a candidate he had lost in all the polls in his city 2 years earlier. PT played to lose…
RT @LulaOficial: Florianópolis with Lula! #Team Lula

📸: @ricardostuckert https://t.co/WIFyL8prrQ
@RakNew_ @kuetubi How dare you. Lock account from yesterday
@geraldoalckmin @LulaOficial Hmm, and in the end you took food from children. bitch hypocrisy
RT @Biakicis: The Northeast is @jairbolsonaro . Today in Caruaru/Pe. There are already journalists doing mental juggling, noting the imminent defeat…
#BTS's #JIMIN declared his support for @LulaOficial it's about that and it's all right.

#WirhYou https://t.co/355Ci5YKQH
RT @jehava1999: Bolsonaro brought more people together in London than squid in Brazil 🤷‍♀️🤣 https://t.co/UPXabc7auy
@anna_bella1900 @Abgf_Brasil Lula disagrees with you... 🤣🤣🤣
@jornalismojoao @cabiusi Obviously it will be #BolsonaroNoPrimeiroTurno22 https://t.co/jXQeRiqdVl
R

KeyboardInterrupt: 